<a href="https://colab.research.google.com/github/trent129/FantasyFootball/blob/league-history/AllTime_H2H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

League Information:

16 teams every year except 2013 and 2014.
17 games in 2007. 14 for 21 now because season is current. 

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16] 07,08,09,10,11,12


[1,2,3,4,5,6,7,9,10,11,12,13,14,15,16] 13,14


1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17] 15,16,17,18,19,20,21

In [ ]:
import pandas as pd
import numpy as np
import requests
import numpy as np
pd.set_option('display.max_rows',2000)

#Input League Data
league_id=441871
seasons=list(range(2007,2022))
espn_s2="AEAycjy2stc8WJHoYThLkYwCRWA3SfH0BQVqpVZgQht%2BQ1jpaPoHZKhRX%2F8gXAcS2uL0NmpBLg49eE90Ss%2B178NmVgNAq2pniAUvPYaUxTkdZ2I18iFtYlgnP%2BqS%2BNTMvrywMdPscNnyOgZdklNmIKwUerecVrDiCENmj46B3%2B0keJGh1MJhHCRrN2VHsSVLnAlGTt6WKglqyBRJ7TDiOz2lJeLHfsFenQgi%2BzL7%2FVBnWCWMB3rUil5pxfgUUZKMMKji1ZyNioWbtPZze%2Fhu8If7Vn542zNqMdF%2FuL4vwSImuw%3D%3D"
swid="{6F956B16-C8CA-4EEA-8B4E-570F9C2B55BD}"
espn_data=pd.DataFrame()
dfs=['df'+str(i) for i in seasons]

for season,df in zip(seasons,dfs):

  #team count
  if season > 2012 and season < 2015 :
    team_count = 15
  
  else:
    team_count = 16
 
  #url for the request
  if season > 2020:
    url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
          str(season) + '/segments/0/leagues/' + str(league_id)

    #API request to get team names
    r = requests.get(url,"view=mTeam",cookies={"swid": swid,  "espn_s2": espn_s2})
    e =r.json()

    #API request to get matchup data  
    r = requests.get(url,"view=mMatchup",cookies={"swid": swid,  "espn_s2": espn_s2})
    d =r.json()

  else:
    url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
          str(league_id) + "?seasonId=" + str(season)
  

    r = requests.get(url,"view=mTeam",cookies={"swid": swid,  "espn_s2": espn_s2})
    e =r.json()[0]

    #API request to get matchup data  
    r = requests.get(url,"view=mMatchup",cookies={"swid": swid,  "espn_s2": espn_s2})
    d =r.json()[0]
  

  #Get Team Names 
  if season < 2013:
    teamnames=[e['teams'][i]['location']+' '+e['teams'][i]['nickname'] for i in range(team_count)]
    tmnames= pd.Series(data=teamnames,index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

  elif season > 2014:
    teamnames=[e['teams'][i]['location']+' '+e['teams'][i]['nickname'] for i in range(team_count)]
    tmnames= pd.Series(data=teamnames,index=[1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17])
    
  else:
    teamnames=[e['teams'][i]['location']+' '+e['teams'][i]['nickname'] for i in range(team_count)]
    tmnames= pd.Series(data=teamnames,index=[1,2,3,4,5,6,7,9,10,11,12,13,14,15,16])


  #Create Points Dataframe
  #Number of Weeks 
  if (season < 2008) or (season ==2021):
    week=17
    
  else: 
    week=16

  
  for each in d['schedule']:
      if 'away' not in each.keys():
          each.update({'away': {'teamId':"None", 'totalPoints':0}})

  gamecount = 8*week +1
  
  df = pd.DataFrame()
  df['Season'] = [season for i in range(gamecount)]
  df['Week'] = [d['schedule'][i]['matchupPeriodId']for i in range(gamecount)]
  df['HomeTeam'] =[tmnames[d['schedule'][i]['home']['teamId']] for i in range(gamecount)]
  df['Opponent'] = [tmnames[d['schedule'][i]['away']['teamId']] if d['schedule'][i]['away']['teamId'] != "None" else '' for i in range(gamecount)]
  df['PF'] = [d['schedule'][i]['home']['totalPoints'] for i in range(gamecount)]
  df['PA'] = [d['schedule'][i]['away']['totalPoints'] for i in range(gamecount)]
 
  
  espn_data=espn_data.append(df).reset_index(drop=True)




In [ ]:
tm=espn_data['HomeTeam'].unique()


In [ ]:
tm[0:15]

array(["Arby's Big Montana", 'Mexico City Hoodlums', 'Pigpile Pillagers',
       'New York Ball Droppers', 'Root Beer Tapper', 'Costa Mesa Rasta',
       'Granite Bay Gangbangers', 'Team Mclovin', 'Sandpoint Seagulls',
       'Bavarian Wolfpack', 'Team ward', 'Vegas High Rollers',
       'North Dakota SlayRide', 'Sideline Shakedown',
       "Flash's Furious Luchadores"], dtype=object)